<a href="https://colab.research.google.com/github/ednavivianasegura/ERAP_Curso_R/blob/main/ERAP_R_Course_ModuloII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Título**:  Manipulación de datos con R  - Módulo II (Manejo de datos desordenados y falta de información )                               
                                          
**Autor(es)**:  Edna Viviana Segura Alvarado - Hans Mauricio Carrillo Hernández

**Fecha**: 2025-06

**Institución**: Universidad de La Rioja    

# ¿Qué son los datos faltantes (Missing Data)?

En R, los valores no disponibles o faltantes se representan con NA (Not Available en inglés) y pueden surgir por múltiples motivos:

- errores de registro
- fallos de instrumentos
- no respuesta en encuestas
- entre otros.
    

Bibliografía:

  Little, Roderick J. A, and Donald B Rubin. Statistical Analysis with Missing Data. Third edition. vol. 793. Newark: Wiley, 2020. Print.


**¿Cómo afectan los datos faltantes a las operaciones básicas?**

Los valores NA en R representan información que no está disponible o que no pudo ser medida. Su presencia puede tener efectos importantes y silenciosos en cálculos básicos si no se gestionan adecuadamente.

In [ ]:
x <- c(5, 8, NA, 10)
mean(x)       # Resultado: NA
sum(x)        # Resultado: NA
sd(x)         # Resultado: NA

**¿Por qué?**

R considera que si hay un NA, no puede estar seguro del resultado total.

Prefiere devolver NA como advertencia de que el cálculo está incompleto.

Por lo tanto, existe el atributo que nos ayuda con esto:

```
na.rm=TRUE
```





In [ ]:
mean(x, na.rm = TRUE)  # Resultado: 7.666...
sum(x, na.rm = TRUE)   # Resultado: 23
sd(x, na.rm = TRUE)    # Resultado: Desviación estándar de los valores observados

**Funciones lógicas y filtros**

Las expresiones lógicas que involucran NA también pueden dar resultados inesperados:

In [ ]:
x <- c(1, 2, NA, 4)
x > 2

Funciones agregadas por grupo **(aggregate, group_by + summarise)**

En dplyr, ocurre lo mismo:

In [ ]:
library(dplyr)

df <- data.frame(grupo = c("A", "A", "B", "B"), valor = c(10, NA, 5, 8))

df %>%
  group_by(grupo) %>%
  summarise(media = mean(valor))  # Resultado: NA para grupo A (el que contiene NAs)

df %>%
  group_by(grupo) %>%
  summarise(media = mean(valor, na.rm = TRUE))  # Ignora los NA correctamente


**Modelado estadístico (regresiones, clustering, PCA...)**

Los modelos no se ajustan correctamente, o no representan la veracidad si no se gestionan los NA:

In [ ]:
set.seed(123)  # Para reproducibilidad

# Generar 100 observaciones
n <- 100
x <- rnorm(n, mean = 50, sd = 10)          # Variable predictora
y <- 6.5 * x + rnorm(n, mean = 0, sd = 12) # Variable respuesta con ruido

plot(x,y)

datos <- data.frame(x = x, y = y)
head(datos)

# Modelo con los datos originales
modeloOriginal <- lm(y ~ x, data = datos)
summary(modeloOriginal)

# Introducir NA aleatoriamente en 15 valores de y
na_indices <- sample(1:n, size = 20)
datos$y[na_indices] <- NA

set.seed(456)  # Para reproducibilidad

# Introducir NA aleatoriamente en 5 valores de x
na_indices2 <- sample(1:n, size = 5)
datos$x[na_indices2] <- NA



summary(datos)
colSums(is.na(datos))

#Intentamos ajustar el modelo
modelo_con_na <- lm(y ~ x, data = datos)
summary(modelo_con_na)

#R ajusta el modelo usando solo las filas donde y no es NA (complete-case analysis).

#Comprobémoslo:
nrow(datos)                # 100 observaciones originales
nrow(modelo_con_na$model)  # Solo usa filas sin NA

#Miremos que pasa con el modelo al imputar, por ejemplo con la media
datos_imputados <- datos
datos_imputados$x[is.na(datos_imputados$x)] <- mean(datos$x, na.rm = TRUE)
datos_imputados$y[is.na(datos_imputados$y)] <- mean(datos$y, na.rm = TRUE)

modelo_imputado <- lm(y ~ x, data = datos_imputados)
summary(modelo_imputado)

nrow(datos_imputados)                # 100 observaciones originales
nrow(modelo_imputado$model)  # Solo 85 filas usadas (15 con NA en x)




## Detección y diagnóstico de valores faltantes

**¿por qué es tan crítico?**

Un solo NA sin tratar puede hacer que todo un análisis falle o se distorsione.

Los datos faltantes deben ser detectados, visualizados y tratados adecuadamente.

El primer paso siempre debe ser saber si alguna(s) de la(s) variables(s) contiene datos faltantes, para esto contamos con muchas herramientas, veremos algunas de ellas:

In [ ]:
# Función para instalar si no está instalado
instalar_si_no <- function(paquete) {
    if (!requireNamespace(paquete, quietly = TRUE)) {
    install.packages(paquete, repos = "https://cloud.r-project.org")
  }
  library(paquete, character.only = TRUE)
}

# Lista de paquetes a verificar
paquetes <- c("naniar", "visdat", "VIM", "remotes","tidyr", "ggplot2")

# Instalar y cargar todos
invisible(lapply(paquetes, instalar_si_no))
remotes::install_github("davidbiol/empire")

In [ ]:
library(naniar)
library(VIM)

Métodos para detectar NAs.

In [ ]:
cat("\n----- Para ver si hay NA --------")

summary(datos)

cat("\n---- Número total de NA ---------")

sum(is.na(datos))

cat("\n----- Número de NA por columna --------")

colSums(is.na(datos))

cat("\n---- Número de NA por fila ---------")

rowSums(is.na(datos))

cat("\n--- Devuelve TRUE si hay al menos un NA en el data.frame. Es la forma más rápida y directa. ----------")

anyNA(datos)

cat("\n---- Devuelve una matriz lógica del mismo tamaño que datos, indicando con TRUE dónde hay NA. ---------")

is.na(datos)

cat("\n---- Devuelve un vector lógico que indica qué filas no tienen ningún NA. Puedes usarlo para filtrar: ---------")

complete.cases(datos)

cat("\n---- Muestra número y porcentaje de NA por fila ---------")

miss_case_summary(datos)

Métodos de visualización:

In [ ]:
print(" -------   Muestra número y porcentaje de NA por variable. -------------")

# La función miss_var_summary(df) pertenece al paquete naniar de R, que se utiliza para analizar y visualizar datos faltantes de manera eficiente y clara.
# miss_var_summary(df) genera un resumen por variable (columna) del número y porcentaje de valores faltantes (NA) en el data frame df.
# y devuelve una tabla resumen con tres columnas principales:
# variable: el nombre de la columna del data frame.
# n_miss: cuántos valores faltantes tiene esa columna.
# pct_miss: el porcentaje de valores faltantes respecto al total de filas.

miss_var_summary(df)

print("---- Muestra un gráfico de barras con la cantidad de valores faltantes por variable y patrones comunes. ---------")

aggr(df, numbers=TRUE)

## Patrones de pérdida de datos

Los patrones describen *dónde* faltan los datos en el dataset. Los principales son:

<center>
 <img src="https://github.com/ednavivianasegura/AccesoImages/blob/main/datossinNA.png?raw=true" alt="descriptiva" width="50%" height="50%">  
</center>


- **Univariado**: solo una variable tiene NA:

<center>
 <img src="https://github.com/ednavivianasegura/AccesoImages/blob/main/UnivariadoNA.png?raw=true" alt="descriptiva" width="50%" height="50%">  
</center>

Solo una variable contiene valores faltantes; el resto de las variables están completamente observadas.
Suele ser el patrón más simple y fácil de manejar.

En este ejemplo vemos la variable Clase tiene valores faltantes. Generalmente es la variable dependiente la que tiene valores faltantes. Probablemente puede presentarse que la persona que clasificó por largo, ancho y forma, olvidó o pasó por alto clasificar esas dos.

- **Multivariado (general)**:

<center>
 <img src="https://github.com/ednavivianasegura/AccesoImages/blob/main/MultivariadoNA.png?raw=true" alt="descriptiva" width="50%" height="50%">  
</center>


Varias variables contienen valores faltantes, sin seguir un orden estructurado. Es el patrón más común en la práctica.

Por ejemplo en un estudio poblacional, la edad, peso e ingresos tienen valores faltantes, pero distribuidos de forma irregular.

Suele representar más dificultad en la aplicación de algunos métodos clásicos.

Vemos en el ejemplo que, si se pierde en una variable, también se pierde en otra para el mismo inidviduo. Es decir, no es aleatorio, tiene un patrón, lo que es muy frecuente en las encuestas.

- **Monótono**: si hay NA en una variable, también en todas las siguientes.

<center>
 <img src="https://github.com/ednavivianasegura/AccesoImages/blob/main/MonotonoNA.png?raw=true" alt="descriptiva" width="50%" height="50%">  
</center>

Hay un orden específico:
si una observación tiene un valor faltante en la variable Xj, entonces tendrá faltantes en todas las variables posteriores (Xj+1, Xj+2, ...).
Suele observarse en estudios longitudinales, estudios que se realizan a través del tiempo, es decir, podemos verlo como una acumulación de la pérdida de información.


- **Arbitrario**: los valores faltantes aparecen en cualquier lugar, sin estructura visible ni lógica aparente. Es el caso más difícil de manejar y el más común. Podemos pensarlo como que la probabilidad de que cada variable tenga un dato faltante, sea la misma para todas las variables.

<center>
 <img src="https://github.com/ednavivianasegura/AccesoImages/blob/main/GeneraNA.png?raw=true" alt="descriptiva" width="50%" height="50%">  
</center>



Este patrón suele requerir de métodos de imputación más robustos.


### Mecanismos de pérdida de datos

Mientras que los patrones de pérdida describen dónde faltan los datos, los mecanismos de pérdida explican por qué faltan. Esta distinción es central en el enfoque de Little & Rubin (2002) y define si los métodos de tratamiento serán válidos o introducirán sesgos.

Se clasifican en tres tipos principales:

1. MCAR — Missing Completely At Random (pérdida completamente al azar)

La probabilidad de que un dato esté ausente es completamente aleatoria y no depende de ninguna variable observada ni no observada.

**Por ejemplo:**

Un sensor de temperatura falla esporádicamente debido a una interferencia eléctrica aleatoria.
El fallo no depende ni del valor real ni de ninguna otra variable.

Qué consecuencias puede traer este tipo de mecanismo:

- Es el caso más benigno.
- El análisis en los datos completos no introduce sesgo.
- Se pueden eliminar filas sin problema (listwise deletion, la veremos más adelante).
- Es poco frecuente en la práctica.

2. MAR — Missing At Random (pérdida al azar)

La probabilidad de que un dato falte depende de otras variables observadas, pero no del valor faltante en sí mismo.

**Por ejemplo:**

Los individuos con más dinero tienden a no reportar su nivel de ingreso.
→ El ingreso falta, pero depende de una variable observada (por ejemplo el estrato, cuyo valor si se tiene).

Qué consecuencias puede traer este tipo de mecanismo:

- Más realista y frecuente.
- El sesgo se puede corregir si incluimos las variables relacionadas en el modelo de imputación (como el estrato en este ejemplo).
- Métodos como imputación múltiple, regresión, o modelos con pesos son válidos.

3. MNAR — Missing Not At Random (Pérdida no al azar)

La probabilidad de que falte un dato depende del valor faltante en sí mismo.

**Por ejemplo:**

Personas con altos ingresos se niegan a declarar su salario por privacidad.
→ El dato falta precisamente porque es alto (digamos que también se negaron a responder la pregunta relacionada con el estrato socioeconómico, por lo que estaría faltando, pero sabríamos porqué).

En resumen, este mecanismo de pérdida de información depende tanto de los datos observados como de los no observados (de los perdidos)

Qué consecuencias puede traer este tipo de mecanismo:

- Este es el caso más problemático.
- No se puede resolver solo con los datos disponibles.
- Se necesitan suposiciones fuertes, modelos de sensibilidad o diseño adicional (como encuestas complementarias).





### Ejemplo práctico:

<center>
 <img src="https://github.com/ednavivianasegura/AccesoImages/blob/main/collage-mamiferos-cabecera.jpg?raw=true" alt="descriptiva" width="50%" height="50%">  
</center>

Veamos esto de una manera práctica con el dataset `sleep`. Este conjunto se basa en una recopilación de datos de sueño en mamíferos que apareció en el estudio R. M. Siegel (2005). Clues to the functions of mammalian sleep. Nature, 437(7063), 1264–1271. Cada fila representa una especie de mamífero. Y las variables describen: características biológicas (peso corporal, peso cerebral), comportamientos relacionados con el sueño (horas REM y no REM), estrategias de supervivencia (depredador, exposición al peligro) y riesgos ecológicos.

| Variable   | Tipo                | Descripción breve                                        |
| ---------- | ------------------- | -------------------------------------------------------- |
| `BodyWgt`  | numérico            | Peso corporal en kilogramos                              |
| `BrainWgt` | numérico            | Peso del cerebro en gramos                               |
| `NonD`     | numérico            | Horas de sueño no REM (sin movimientos oculares rápidos) |
| `Dream`    | numérico            | Horas de sueño REM (con sueños)                          |
| `Sleep`    | numérico            | Total de horas de sueño por día                          |
| `Span`     | numérico            | Esperanza de vida en años                                |
| `Gest`     | numérico            | Duración de la gestación en días                         |
| `Pred`     | categórica (factor) | ¿Es el animal un depredador? (categoría: sí/no)          |
| `Exp`      | categórica (factor) | ¿Está expuesto a peligro? (bajo, medio, alto)            |
| `Danger`   | numérico            | Nivel de peligro general (valor numérico del riesgo)     |



In [ ]:
data(sleep)
# Ver dimensiones del dataset
dim(sleep)
 #Ver variables del dataset
str(sleep)
glimpse(sleep)

In [ ]:
View(sleep)

Hacemos primero un análisis descriptivo

In [ ]:
summary(sleep)

Dentro del análisis inicial, es importante revisar la correlación entre las variables

In [ ]:
#Correlación (usa una función del paquete visdat para visualizar gráficamente una matriz de correlación entre las variables del dataset sleep:)
visdat::vis_cor(sleep, na_action = "complete.obs")
#El operador :: se utiliza para acceder a una función específica dentro de un paquete, sin necesidad de cargar todo el paquete con library().
#na_action = "complete.obs", indica cómo manejar los valores faltantes (NA) al calcular las correlaciones,
# significa que sólo se usan aquellas filas (observaciones) que no tienen ningún NA en ninguna de las variables consideradas.



Si existe algún tipo de correlación, nos facilita la tarea de estimar los datos faltantes de alguna manera más precisa.

A continuación visualizamos los datos faltantes:

In [ ]:
?VIM::aggr

In [ ]:
# Visualizar patrón general

#Gráficas
visdat::vis_dat(sleep, sort_type = FALSE)
#¿Qué hace?
# Genera un mapa visual del dataset, donde:
# Cada fila representa una observación.
# Cada columna representa una variable.
# Los colores indican el tipo de dato (numérico, carácter, lógico, NA, etc.).
# Argumento:
# sort_type = FALSE: mantiene el orden original de las variables.
# (Si fuera TRUE, ordenaría las columnas por tipo de dato.)
# ¿Para qué sirve?
# Ver de forma rápida qué variables tienen NA y de qué tipo son las demás.
# Identificar valores atípicos o inconsistencias de tipo (por ejemplo, una columna que debería ser numérica pero es carácter).


visdat::vis_miss(sleep)
#Este nos ayuda a identificar los patrones:
# A qué patrón parece?
# Vemos que cuando se pierde NonD, se pierde también Dream, si comprobamos
# la correlación, vemos que no es tan alta.
# Puede que sea una simple casualidad.

VIM::aggr(sleep, numbers = TRUE, sortVars = TRUE, gap = 3,prop = TRUE,
     ylab = c("Frecuencia de NA", "Patrón de pérdida"))


#Produce una visualización compuesta que incluye:
# Un histograma con el porcentaje de valores faltantes por variable (izquierda).
# Una matriz de combinaciones de ausencia/presencia entre variables (derecha).
# Argumentos clave:
#numbers = TRUE: muestra los valores numéricos sobre las barras.
#sortVars = TRUE: ordena las variables de mayor a menor porcentaje de NA.
#gap = 3: espacio entre los gráficos izquierdo y derecho.
#prop = TRUE: muestra proporciones en lugar de conteo absoluto.
#ylab: etiquetas personalizadas del eje y.
#¿Para qué sirve?
#Diagnóstico visual completo del patrón de pérdida de datos
# Permite ver cómo esta la proporción de pérdida en cada una de las variables.
#Permite detectar si hay correlaciones entre variables faltantes (por ejemplo, cuando dos columnas tienden a estar ausentes juntas).





¿Qué podemos ver aquí?

Gráfico aggr() — Frecuencia y patrón de pérdida
Este gráfico tiene dos componentes:

- A la izquierda: barras verticales rojas
Representan el porcentaje de valores faltantes por variable.

De acuerdo con la tabla y el gráfico de barras:

| Variable | % NA   | Significado                                               |
|----------|--------|------------------------------------------------------------|
| NonD     | 22.6%  | Horas de sueño no REM faltantes en 22.6%                   |
| Dream    | 19.4%  | Horas de sueño REM faltantes en 19.4%                      |
| Sleep    | 6.5%   | Total de horas de sueño faltantes                          |
| Span     | 6.5%   | Esperanza de vida faltante                                 |
| Gest     | 6.5%   | Días de gestación faltantes                                |
| Resto    | 0%     | Completas: sin valores faltantes                           |


Vemos entonces que el mayor problema está en las variables NonD y Dream.

- A la derecha: matriz de patrones de NA
Cada columna representa una variable.

Cada fila representa una combinación específica de presencia/ausencia de datos en una observación.

Rojo = valor faltante | Azul = valor presente

¿Qué observamos?

Hay una gran cantidad de combinaciones diferentes de valores faltantes, lo que sugiere un patrón de pérdida general (arbitrario), no monótono.

Concretamente:

- Fila completamente azul con valor 0,67, indica que todas las variables están presentes (sin NA).
Es decir, ese patrón representa las observaciones completas o casos completos: Valor 0,67: significa que el 67% de las observaciones del dataset están completas, es decir, no tienen ningún valor faltante en ninguna variable.
Por lo tanto, en el dataset sleep con 62 observaciones, esto corresponde a aproximadamente 42 observaciones completas (62*0,67)

- Algunas observaciones tienen múltiples NA; otras tienen uno solo, por ejemplo:

Fila con rojo solo en Gest y valor 0,048, representa el patrón en el que solo falta la variable Gest, el resto de variables está presente. El valor 0,048: implica que el 4.8% de las observaciones tienen NA únicamente en Gest (0,048*65 aprox 3 observaciones).



In [ ]:
#Información de número de datos faltantes totales y de la posición
empire::count_miss(data = sleep) #Número de datos faltantes
empire::pos_miss(data = sleep) #Posición fila-columna de los datos faltantes

## Técnicas de manejo de datos faltantes.

| Estrategia                     | Descripción breve                              | Requiere suposiciones fuertes     | Mantiene tamaño de muestra |
| ------------------------------ | ---------------------------------------------- | --------------------------------- | -------------------------- |
| Eliminación de casos           | Omitir filas con NA                            | ❌ Solo válida si MCAR (resultados un poco decentes)             | ❌  (producir sesgos)                        |
| Imputación con media/mediana   | Reemplaza NA por un valor fijo                 | ✅ Simplista, distorsiona varianza | ✅                          |
| Imputación por regresión       | Predice el NA con otras variables observadas   | ✅ Supone relación lineal          | ✅                          |
| Imputación con penalización    | Como la regresión, pero con regularización     | ✅ Menos riesgo de sobreajuste     | ✅                          |
| Imputación múltiple / avanzada | Genera varios datasets imputados + combinación | ✅ Compleja, pero más rigurosa     | ✅                          |


### Eliminación de casos
Es fácil y sin sesgo si los datos faltan al azar.
Puede eliminar mucha información.


In [ ]:
# Eliminar filas con cualquier NA

View(sleep)

sleep_limpio <- sleep[complete.cases(sleep), ]
print(dim(sleep_limpio))

sleep_limpio2<- sleep %>%
  filter(if_all(everything(), ~ !is.na(.))) ## Filtrar las filas que NO tengan ningún valor NA en ninguna de sus columnas.
  # 'if_all(everything(), ~ !is.na(.))' verifica que todas las columnas de cada fila no tengan NA
  # everything(): selecciona todas las columnas del data frame.
  # if_all(...): evalúa si se cumple una condición en todas las columnas.
print(dim(sleep_limpio2))

# Ver resumen del nuevo dataset
summary(sleep_limpio)
View(sleep_limpio) #ver los elementos eliminados

# Ver cuántas filas se eliminaron (cantidad y proporción)
cat("\nSe han eliminado", nrow(sleep) - nrow(sleep_limpio),"observaciones, que equivalen aproximadamente al",round((abs(nrow(sleep) - nrow(sleep_limpio))/nrow(sleep)*100),2),"% de la información total")



### Imputación con la media

- Sustiye el dato faltante por el valor medio de la variable, calculado con los datos existentes.
- Es especialmente válida cuando los datos faltantes siguen un patrón MCAR, cuando la correlación entre las variables es baja y el porcentaje de datos faltantes es bajo.
- Afecta la potencia de la prueba.
- Afecta la distribución de las variables.
- No tiene en cuenta las demás variables.

In [ ]:
#defino función para calcular sd a todas las variables numércias:
st_total<- function(x)
    {if(is.numeric(x)) resultado <- sd(x, na.rm = TRUE) else resultado <-NA
    return (resultado)}

cat("\nVista de datos originales")
View(sleep) #los nuevos datos

# Imputación con la media
new_sleep_mean <- empire::impute_mean(data = sleep)
#tiene varios atributos (vario s objetos)

#new_sleep_mean$positions #muestra las posiciones de los valores faltantes
#new_sleep_mean$imp_values #cuales son los valores imputados


cat("\nVista de datos nuevos")
View(new_sleep_mean$new_data) #los nuevos datos

# Revisar variables
# Comparación de datasets (medias, cuartiles y desvest)

cat("\nResumen datos normales")
summary(sleep)
sapply(sleep,st_total)



cat("\nResumen datos imputados con la media")
summary(new_sleep_mean$new_data)
sapply(new_sleep_mean$new_data, st_total)




### Imputación con la mediana

- Sustiye el dato faltante por la mediana de la variable, calculado con los datos existentes.
- Es especialmente válida cuando los datos faltantes siguen un patrón MCAR, cuando la correlación entre las variables es baja y el porcentaje de datos faltantes es bajo.
- Es mejor que la media, en datos no paramétricos, mientras que si los datos son paramétricos es mejor la media que la mediana.
- Afecta la distribución de la variable (tanto media como mediana)
- No tiene en cuenta las demás variables.

In [ ]:
#Imputación con la mediana
new_sleep_mediana <- empire::impute_median(data = sleep)

#new_sleep_mediana$imp_values
#new_sleep_mediana$new_data


# Revisar variables
# Comparación de datasets (medias, cuartiles y desvest)

cat("\nResumen datos normales")
summary(sleep)
sapply(sleep,st_total)

cat("\nResumen datos imputados con la mediana")
summary(new_sleep_mediana$new_data)
sapply(new_sleep_mediana$new_data, st_total)


Ahora veremos técnicas más avanzadas, más cercanas a la realidad (mejores estimación):

### Estimación por regresión lineal múltiple

- La imputación del dato faltante en la variable de interés depende de otras variables.
- Reemplaza cada dato faltante por un valor estimado basado en un modelo de regresión con las variables existentes.
- La técnica es relativamente efectiva cuando las correlaciones de las variables son estables, es decir, no varían mucho con un dato más o un dato menos (si tenemos un tamaño muestral relativamente grande).
- Si se cumplen los supuestos de una regresión múltiple, homocedasticidad, linealidad, etc, estqa técnica es muy muy buena.
- Esta técninca evita alterar significativamente las desviaciones estándar o en general, la forma de la distribución.

(volver al ejemplo de patrón univariado (imagen arriba))


In [ ]:
cor(sleep[, 1:7], use = "pairwise.complete.obs")

In [ ]:
# Estimación por regresión lineal múltiple
# Solo seleccionamos variables numéricas para imputar por regresión

empire::estimate_mlr(data = sleep[,1:7])
# En el momento de aplicar la regresión lineal múltiple, los parámetros se estiman calculando X*transpuesta de X y luego se calcula la inversa.
# Lo que pasa es que en algún momento del cálculo debe estar dando "indefinido" al hacer este cálculo.
# Este problema, con la estimación lineal múltiple, sucede cuando se viola alguno de los supuestos, en este caso, lo más probable es que se esté
# violando el supuesto de multicolinealidad


In [ ]:
#En este caso, sólo con fines de mostrar los resultados, escogemos variables que no
# presenten multicolinealidad, pero veremos como solucionarlo de una manera más formal en la siguiente técnica.


#Asumimos cumplidas las hipótesis de linealidad, normalidad multivariada, no multicolinealidad y homocedasticidad:
imp_mlr <- empire::estimate_mlr(data = sleep[, c("BodyWgt", "NonD", "Span", "Gest")], diff= 10e-08)

# Esta función itera. Estima los datos faltantes, con base en la regresión lineal múltiple, luego vuelve a aplicar la
# rlm con los datos nuevos, calcula la diferencia... varias veces hasta cuando la diferencia sea muy pequeña (menor a 10e-08)
# Ver resultados
imp_mlr$positions
imp_mlr$est_values
imp_mlr$new_data



# Revisar variables
# Comparación de datasets (medias, cuartiles y desvest)

cat("\nResumen datos normales")
summary(sleep[, c("BodyWgt", "NonD", "Span", "Gest")])
sapply(sleep[, c("BodyWgt", "NonD", "Span", "Gest")],st_total)

cat("\nResumen datos imputados con la media")
summary(imp_mlr$new_data)
sapply(imp_mlr$new_data, st_total)


# empire::estimate_mlr() realiza una imputación secuencial:
# Imputa primero las variables con menos NA.
# Luego usa esas variables ya imputadas como predictores para las siguientes.
# Si una variable explicativa tiene NA en el momento del modelo, solo se usan las filas completas para ajustar y predecir.
# Pero: si muchas variables tienen NA simultáneamente, puede fallar o devolver resultados sesgados.





### Regresión lineal múltiple penalizada (RIDGE - Estimation of Missin-Data using Penalized Iterative Regressions, de la librería empire)

empire es un paquete de R que proporciona una serie de funciones para trabajar la imputación (estimación) de datos faltantes:

- count_miss
- pos_miss
- impute_mean
- impute median
- estimate_mlr
- estimate_ridge (la más importante)
    - Aplica la penalización (regularización) de Ridge a la regresión.
        La regresión penalizada ridge permite imputar valores faltantes de forma más robusta que la regresión clásica. Esto se logra añadiendo una penalización que estabiliza las estimaciones cuando hay muchas variables correlacionadas o faltan muchas observaciones al mismo tiempo. La función estimate_ridge() del paquete empire automatiza este proceso y devuelve un dataset limpio sin NA.

In [ ]:
# Estimación por regresión lineal múltiple penalizada
new_sleep_ridge <- empire::estimate_ridge(data = sleep[,1:7], diff = 10, ridge_alpha = 0)

#vemos que ahora no sacó ningún error

new_sleep_ridge$est_values #valores estimados que están teniendo en cuenta las otras variables
new_sleep_ridge$new_dat

# Revisar variables
# Comparación de datasets (medias, cuartiles y desvest)

cat("\nResumen datos normales")
summary(sleep)
sapply(sleep,st_total)

cat("\nResumen datos imputados con la media")
summary(new_sleep_ridge$new_data)
sapply(new_sleep_ridge$new_data, st_total)

Finalmente creamos una comparación entre métodos:

In [ ]:
# Comparación de métodos de imputación sobre el dataset `sleep`

# Seleccionar solo las columnas numéricas para aplicar métodos de imputación numérica
sleep_numericas <- dplyr::select_if(sleep, is.numeric)

# Eliminar filas que tengan al menos un NA (eliminación de casos)
sleep_limpio <- sleep[complete.cases(sleep), ]

# Aplicar imputación por la media a columnas numéricas
new_sleep_mean <- empire::impute_mean(data = sleep_numericas)

# Aplicar imputación por la mediana a columnas numéricas
new_sleep_median <- empire::impute_median(data = sleep_numericas)

# Aplicar imputación por regresión lineal penalizada (ridge) a primeras 7 columnas numéricas
new_sleep_ridge <- empire::estimate_ridge(data = sleep[,1:7], diff = 10, ridge_alpha = 0)

# ------------------- Definir función para alinear datasets -------------------
# Esta función unifica el número de columnas y añade el nombre del método como etiqueta
alinear_datos <- function(df, nombre, columnas_ref) {
  if (is.list(df) && !is.null(df$new_data)) {
    df <- df$new_data  # Extraer los datos imputados si el objeto es una lista
  }
  faltantes <- setdiff(columnas_ref, colnames(df))  # Identificar columnas faltantes
  df[faltantes] <- NA  # Añadirlas con NA si faltan
  df <- df[, columnas_ref]  # Reordenar columnas según las de referencia
  df$metodo <- nombre       # Añadir columna que indica el método
  return(df)
}


# Definir las columnas de referencia (del dataset original)
columnas_ref <- colnames(sleep)

# Unir todos los datasets con imputaciones diferentes en un solo data frame
# bind_rows() concatena por filas

todos <- bind_rows(
  alinear_datos(sleep, "original (con NA)", columnas_ref),
  # 1. Usa el conjunto original 'sleep' (con valores faltantes)
  #    y lo etiqueta como "original (con NA)" añadiendo una columna 'metodo'
  #    Además, lo ajusta para que tenga las mismas columnas y orden que 'columnas_ref'

  alinear_datos(sleep_limpio, "eliminación", columnas_ref),
  # 2. Usa la versión sin valores faltantes ('sleep_limpio'),
  #    resultado de eliminar filas con NA (complete.cases)
  #    Se etiqueta con 'metodo = "eliminación"'

  alinear_datos(new_sleep_mean$new_data, "media", columnas_ref),
  # 3. Usa los datos imputados con la media y los etiqueta como "media"

  alinear_datos(new_sleep_median$new_data, "mediana", columnas_ref),
  # 4. Usa los datos imputados con la mediana y los etiqueta como "mediana"

  alinear_datos(new_sleep_ridge$new_data, "ridge", columnas_ref)
  # 5. Usa los datos imputados mediante regresión ridge y los etiqueta como "ridge"
)

cat("\n todos: ")

View(todos)

# Convertir columnas categóricas en factores para asegurar que sean tratadas correctamente
categoricas <- c("Pred", "Exp", "Danger")
todos[categoricas] <- lapply(todos[categoricas], as.factor)

# Asegurar también que las variables categóricas del dataset original sean factores
sleep$Pred <- as.factor(sleep$Pred)
sleep$Exp <- as.factor(sleep$Exp)
sleep$Danger <- as.factor(sleep$Danger)

# ====================== ANÁLISIS NUMÉRICO ======================

# Calcular estadísticas descriptivas (media, sd, min, max) para variables numéricas agrupadas por método
# group_by() agrupa por el método de imputación
# summarise(across(...)) aplica múltiples funciones a columnas numéricas
resumen <- todos %>%                         # 1. Toma el data frame 'todos'
  group_by(metodo) %>%                        # 2. Agrupa las observaciones por la columna 'metodo'
  summarise(                                  # 3. Resume cada grupo con estadísticas numéricas
    across(                                   # 4. Aplica funciones a todas las columnas numéricas
                                              # across() aplica una o más funciones a una o más columnas, dentro de funciones como summarise() o mutate().
      where(is.numeric),                      # 5. Selecciona solo las columnas numéricas
      list(                                   # 6. Aplica una lista de funciones resumen:
        media = ~mean(., na.rm = TRUE),       #   - Media, ignorando NA
        sd = ~sd(., na.rm = TRUE),            #   - Desviación estándar, ignorando NA
        min = ~if (all(is.na(.))) NA          #   - Mínimo (si toda la columna es NA, devuelve NA)
               else min(., na.rm = TRUE),
        max = ~if (all(is.na(.))) NA          #   - Máximo (si toda la columna es NA, devuelve NA)
               else max(., na.rm = TRUE)
      ),
      .names = "{.col}_{.fn}"                 # 7. Nombra las columnas como: variable_función (ej: REM_media)
    ),
    .groups = "drop"                          # 8. Elimina el agrupamiento después del resumen
  )


# Mostrar resumen en consola
cat("\n Muestra el resumen realizado")
print(resumen)

# Convertir a formato largo para graficar comparaciones
# pivot_longer transforma de formato ancho a largo para graficar con ggplot
long <- todos %>%
  pivot_longer(cols = where(is.numeric), names_to = "variable", values_to = "valor")

cat("\n muestra a TODOS en el formato largo para utilizarlo en la visualización")
View(long)

# Visualización numérica: boxplots por variable y método
# geom_boxplot() permite comparar medianas y dispersión

ggplot(long, aes(x = metodo, y = valor, fill = metodo)) +
  # Crea un gráfico usando el data frame 'long'
  # aes(...) define los mapeos estéticos:
  #   - x = metodo: en el eje x se colocan los métodos de imputación
  #   - y = valor: en el eje y los valores de las variables numéricas
  #   - fill = metodo: se colorean los boxplots según el método

  geom_boxplot(outliers = FALSE) +
  # Dibuja un diagrama de caja (boxplot) para cada combinación de método y variable
  # outliers = FALSE → no se muestran los puntos extremos (atípicos)

  facet_wrap(~variable, scales = "free") +
  # Divide el gráfico en paneles independientes, uno por cada variable numérica
  # ~variable indica que cada panel se basa en la variable (columna 'variable')
  # scales = "free" permite que cada gráfico tenga su propio eje y, según su rango

  theme_minimal() +
  # Aplica un tema visual limpio y sencillo al gráfico (fondo blanco sin recuadros)

  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  # Rota las etiquetas del eje x 45 grados para que no se solapen
  # hjust = 1 → alinea las etiquetas hacia la derecha

  labs(title = "Comparación de variables numéricas por método de imputación")
  # Añade un título al gráfico


# Visualización numérica: curvas de densidad
# geom_density() muestra la forma de la distribución
long_filtrado <- long %>% filter(!is.na(valor))

ggplot(long_filtrado, aes(x = valor, color = metodo, fill = metodo)) +
  # Crea un gráfico de densidad a partir del data frame 'long_filtrado'
  # aes(...) define el mapeo estético:
  #   - x = valor: el eje x muestra los valores de la variable numérica
  #   - color = metodo: las curvas de densidad tendrán un contorno coloreado según el método de imputación
  #   - fill = metodo: las curvas también estarán sombreadas según el método (área bajo la curva)

  geom_density(alpha = 0.2) +
  # Dibuja curvas de densidad para comparar distribuciones suaves de los valores numéricos
  # alpha = 0.2 → establece la transparencia del sombreado (para que se vean las superposiciones)

  facet_wrap(~variable, scales = "free") +
  # Divide el gráfico en varios paneles, uno por cada variable numérica
  # ~variable indica que se crea un panel por variable
  # scales = "free" permite que cada panel tenga su propio rango de valores

  theme_minimal() +
  # Aplica un diseño limpio y simple al gráfico (sin fondos ni líneas extra)

  labs(title = "Distribución de densidad por variable y método de imputación")
  # Añade un título general al gráfico


# ====================== ANÁLISIS CATEGÓRICO ======================

# Convertir datos categóricos a formato largo para graficar
# Se elimina 'ridge' porque no contiene variables categóricas
long_cat <- todos %>%
  filter(metodo != "ridge") %>%
  # Filtra el data frame 'todos' para excluir las filas del método "ridge"
  # Esto es necesario porque el método ridge puede haber eliminado o no haber imputado variables categóricas

  select(metodo, all_of(categoricas)) %>%
  # Selecciona únicamente la columna 'metodo' y las variables categóricas definidas en el vector 'categoricas'
  # 'all_of(categoricas)' permite seleccionar variables usando un vector de nombres (por ejemplo: c("Pred", "Exp", "Danger"))

  pivot_longer(cols = -metodo, names_to = "variable", values_to = "valor")
  # Convierte el data frame de formato ancho a largo
  # 'cols = -metodo' → convierte en largo todas las columnas excepto 'metodo'
  # 'names_to = "variable"' → crea una columna llamada 'variable' con los nombres de las variables categóricas
  # 'values_to = "valor"' → crea una columna llamada 'valor' con los valores de cada categoría


# Gráfico de barras para variables categóricas
# geom_bar() con position = "dodge" compara frecuencias absolutas entre métodos

ggplot(long_cat, aes(x = valor, fill = metodo)) +
  # Inicia un gráfico con ggplot usando el data frame 'long_cat'
  # aes(...) define los mapeos estéticos:
  #   - x = valor: en el eje x se colocan las categorías de las variables (por ejemplo, "Sí", "No", "A", "B", etc.)
  #   - fill = metodo: cada barra se colorea según el método de imputación utilizado (media, mediana, etc.)

  geom_bar(position = "dodge") +
  # Dibuja un gráfico de barras con barras una al lado de la otra (posición "dodge")
  # Cada barra representa la frecuencia absoluta de cada categoría según el método

  facet_wrap(~variable, scales = "free") +
  # Divide el gráfico en paneles independientes, uno para cada variable categórica (como "Pred", "Exp", etc.)
  # scales = "free" permite que cada panel tenga su propia escala de conteo en el eje y

  theme_minimal() +
  # Aplica un tema visual limpio y sin bordes pesados

  labs(
    title = "Distribución de variables categóricas por método de imputación",
    x = "Categoría",
    y = "Frecuencia"
  ) +
  # Define el título principal y los nombres de los ejes

  theme(axis.text.x = element_text(angle = 45, hjust = 1))
  # Rota las etiquetas del eje x 45 grados para que no se solapen
  # hjust = 1 → alinea las etiquetas a la derecha



In [ ]:
# @title crea mapa
install.packages("wordcloud")
install.packages("RColorBrewer")

library(wordcloud)
library(RColorBrewer)
paquetes <- c(
  "VIM", "naniar", "missForest", "Amelia", "mice", "Hmisc", "depmixS4",
  "imputeTS", "zoo", "forecast", "softImpute", "MissMDA", "EMCluster",
  "mi", "simputation", "BaylorEdPsych", "Rmagic", "Denoiser", "gapfill",
  "DtImpute", "DrImpute", "Hot", "Circspacetime", "Fastlink", "Eigenmodel",
  "Cassandra", "Experiment", "BootImpute", "Forimp", "FhdI", "ImputeMean"
)
set.seed(123)
frecuencias <- sample(10:100, length(paquetes), replace = TRUE)
wordcloud(
  words = paquetes,
  freq = frecuencias,
  min.freq = 1,
  scale = c(4, 0.8),             # Tamaño de fuente
  colors = brewer.pal(8, "Dark2"),
  random.order = FALSE,
  rot.per = 0.2,                 # Proporción de palabras giradas
  main = "Paquetes de R para datos faltantes"
)




# Ejercicio, repetir el análisis con los datos de mlbench

Conjuntos de datos para Machine Learning:

El paquete mlbench proporciona una colección de datasets clásicos de aprendizaje automático y estadística. Es muy utilizado para realizar pruebas, prácticas y comparaciones de algoritmos de clasificación, regresión y clustering.

Este paquete incluye datos reales y simulados.

Útil para probar algoritmos de aprendizaje supervisado (como árboles de decisión, k-NN, redes neuronales, etc.).

Los conjuntos de datos suelen estar bien documentados y preparados para su uso inmediato.

Ejemplos de datasets incluidos:

**PimaIndiansDiabetes:** información médica para predecir diabetes en mujeres.

BostonHousing: precios de viviendas en Boston en función de variables socioeconómicas.

Sonar: señales sonar para distinguir entre minas y rocas.

BreastCancer: datos para detección de cáncer de mama.

Nosotros utilizaremos el de diabetes:

Este conjunto de datos proviene de un estudio del Instituto Nacional de Diabetes en EE.UU. Su objetivo es predecir si una paciente padece diabetes tipo 2 basándose en mediciones médicas y demográficas.

Es una versión modificada del dataset original (PimaIndiansDiabetes), en la cual:

Se reemplazan los valores inválidos (0) por NA en variables donde el cero no tiene sentido (como presión sanguínea o nivel de glucosa).

Es más adecuado para realizar análisis de datos reales y tratamiento de datos faltantes.

**Descripción de las variables:**

| Variable   | Tipo     | Descripción                                                           |
| ---------- | -------- | --------------------------------------------------------------------- |
| `pregnant` | numérico | Número de embarazos                                                   |
| `glucose`  | numérico | Concentración de glucosa en plasma a 2 horas en una prueba oral       |
| `pressure` | numérico | Presión arterial diastólica (mm Hg)                                   |
| `triceps`  | numérico | Grosor del pliegue cutáneo del tríceps (mm)                           |
| `insulin`  | numérico | Nivel de insulina sérica (mu U/ml)                                    |
| `mass`     | numérico | Índice de masa corporal (BMI) = peso (kg) / (altura (m))²             |
| `pedigree` | numérico | Función del pedigrí de diabetes (influencia genética/familiar)        |
| `age`      | numérico | Edad en años                                                          |
| `diabetes` | factor   | Variable respuesta: `"pos"` = positivo (diabetes), `"neg"` = negativo |




In [ ]:
if (!requireNamespace("mlbench")) install.packages("mlbench")
library(mlbench)


data("PimaIndiansDiabetes2")
heart <- PimaIndiansDiabetes2

Realiza el análisis de las deferencias que hicimos antes, con las otras variables, pressure,	triceps,	insulin,	mass,	pedigree,	age